In [13]:
from dask_jobqueue import SLURMCluster
from distributed import Client
import uproot
from joblib import parallel_backend, Parallel, delayed

In [2]:
cluster = SLURMCluster(cores=1, memory='1GB')

In [3]:
cluster.scale(2)

In [4]:
client = Client(cluster)

In [11]:
client

Client Scheduler: tcp://192.33.123.23:41663 Dashboard: http://192.33.123.23:8787/status,Cluster Workers: 2 Cores: 2 Memory: 2.00 GB


In [14]:
# Define function which is fed to each joblib/Dask worker

def read_branches(file_name, tree_name, branches_query):
    # This does NOT read the branches (yet)
    file = uproot.open(file_name)
    tree = file[tree_name]
    # This reads the branches
    return tree.arrays(filter_name=branches_query)

In [17]:
%%time
# Code incapsulated in parallel_backend is scheduled and waited for
# As can be seen in the dashboard, this is correctly spread over the cluster

file_name = 'flashgg_investigate/test_nanoaod_99_Skim.root'
tree_name = 'Events'
queries = ['Electron*', 'Photon*']
    
with parallel_backend('dask', scheduler_host=cluster):
    electrons, photons = Parallel()(delayed(read_branches)(file_name, tree_name, q) for q in queries)

CPU times: user 183 ms, sys: 20.9 ms, total: 204 ms
Wall time: 6.91 s


In [19]:
electrons.type

1664 * {"Electron_charge": var * int32, "Electron_cleanmask": var * uint8, "Electron_convVeto": var * bool, "Electron_cutBased": var * int32, "Electron_cutBased_Fall17_V1": var * int32, "Electron_cutBased_HEEP": var * bool, "Electron_deltaEtaSC": var * float32, "Electron_dr03EcalRecHitSumEt": var * float32, "Electron_dr03HcalDepth1TowerSumEt": var * float32, "Electron_dr03TkSumPt": var * float32, "Electron_dr03TkSumPtHEEP": var * float32, "Electron_dxy": var * float32, "Electron_dxyErr": var * float32, "Electron_dz": var * float32, "Electron_dzErr": var * float32, "Electron_eCorr": var * float32, "Electron_eInvMinusPInv": var * float32, "Electron_energyErr": var * float32, "Electron_eta": var * float32, "Electron_genPartFlav": var * uint8, "Electron_genPartIdx": var * int32, "Electron_hoe": var * float32, "Electron_ip3d": var * float32, "Electron_isPFcand": var * bool, "Electron_jetIdx": var * int32, "Electron_jetPtRelv2": var * float32, "Electron_jetRelIso": var * float32, "Electron_l

In [20]:
photons.type

1664 * {"Photon_charge": var * int32, "Photon_chargedHadronIso": var * float32, "Photon_cleanmask": var * uint8, "Photon_cutBased": var * int32, "Photon_cutBased_Fall17V1Bitmap": var * int32, "Photon_eCorr": var * float32, "Photon_electronIdx": var * int32, "Photon_electronVeto": var * bool, "Photon_energyErr": var * float32, "Photon_eta": var * float32, "Photon_genPartFlav": var * uint8, "Photon_genPartIdx": var * int32, "Photon_hoe": var * float32, "Photon_isScEtaEB": var * bool, "Photon_isScEtaEE": var * bool, "Photon_jetIdx": var * int32, "Photon_mass": var * float32, "Photon_mvaID": var * float32, "Photon_mvaID_Fall17V1p1": var * float32, "Photon_mvaID_WP80": var * bool, "Photon_mvaID_WP90": var * bool, "Photon_pdgId": var * int32, "Photon_pfRelIso03_all": var * float32, "Photon_pfRelIso03_chg": var * float32, "Photon_phi": var * float32, "Photon_photonIso": var * float32, "Photon_pixelSeed": var * bool, "Photon_pt": var * float32, "Photon_pt_FNUDown": var * float32, "Photon_pt_FN